# App processes cleaning

In [1]:
import sys

sys.path.append('..')

In [2]:
import os
import gc
import yaml
import json
import requests
import numpy as np
import pandas as pd

from pymongo import MongoClient

from utils import get_files, memory_usage, \
correct_encoding, update_page_metadata

In [3]:
BASE_PATH = '../../'

CONFIG_DIR = os.path.join(BASE_PATH, 'config')
STORAGE_DIR = os.path.join(BASE_PATH, 'storage')
DATA_DIR = os.path.join(BASE_PATH, 'data')

config = yaml.load(open(os.path.join(CONFIG_DIR, 'env.yml')),
                   Loader=yaml.FullLoader)

In [4]:
model = 'app_processes'
model_storage = os.path.join(STORAGE_DIR, model)

if not os.path.exists(model_storage):
    os.makedirs(model_storage)
    
client = MongoClient('mongodb://{}:{}@{}:{}'.format(config['MONGO_USERNAME'], config['MONGO_PASSWORD'],
                                                    config['MONGO_HOST'], config['MONGO_PORT']))
metadata_db = client[config['MONGO_DATABASE']]

model_metadata = metadata_db[model]

model_metadata.delete_many({})
model_metadata.insert_one({'pages': []})

In [5]:
csv_files = get_files(os.path.join(DATA_DIR, model, '*.csv'))

In [6]:
if not csv_files:
    raise FileNotFoundError("Couldn't find any csv files! Please make sure the filepath exists")

In [8]:
df = pd.read_csv(csv_files[0], sep=';', nrows=100)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 100 non-null    int64 
 1   sample_id          100 non-null    int64 
 2   name               100 non-null    object
 3   application_label  81 non-null     object
 4   is_system_app      100 non-null    int64 
 5   version_name       81 non-null     object
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


In [15]:
df['name'].str.split(':', n=1, expand=True)

,0,1
0,com.hmatalonga.greenhub,None
1,com.qualcomm.telephony,None
2,com.google.android.music,main
3,com.kms.free,None
4,com.lge.systemserver,None
...,...,...
95,com.kms.free,None
96,com.lge.systemserver,None
97,com.facebook.katana,videoplayer
98,com.facebook.orca,videoplayer
